In [1]:
import os

# Replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key
api_key = ''

# Set the environment variable
os.environ['OPENAI_API_KEY'] = api_key


In [2]:
import os
import pickle

# Set the path to the directory
dir_path = '/Users/hezheng/Downloads/10k'

# Create a dictionary to store the pickle objects
pickle_dict = {}

# Walk through the files in the directory
for filename in os.listdir(dir_path):
    # Ensure we're working with a pickle file
    if filename.endswith('.pkl'):
        with open(os.path.join(dir_path, filename), 'rb') as f:
            # Load the pickle file and store it in the dictionary
            pickle_obj = pickle.load(f)
            pickle_dict[filename] = pickle_obj

# Now, pickle_dict holds all of your loaded pickle objects
# Key is the filename and value is the pickle object


In [3]:
pickle_dict.keys()

dict_keys(['7A.pkl', '9.pkl', '8.pkl', '1A.pkl', '14.pkl', '1B.pkl', '10.pkl', '11.pkl', '13.pkl', '12.pkl', '6.pkl', '9A.pkl', '7.pkl', '5.pkl', '4.pkl', '1.pkl', '3.pkl', '2.pkl'])

In [4]:
print(pickle_dict['1.pkl'].head())

  ticker                                          section1A       date  \
0   aapl   Item 1. Business \n\nCompany Background \n\nT... 2022-09-24   
1   aapl   Item 1. Business \n\nCompany Background \n\nT... 2021-09-25   
2   aapl   Item 1. Business \n\nCompany Background \n\nT... 2020-09-26   
3   aapl   Item 1. \n\nBusiness \n\n##TABLE_END Company ... 2019-09-28   
4   aapl   Item 1. \n\nBusiness \n\n##TABLE_END Company ... 2018-09-29   

       ticker_date  
0  aapl_2022-09-24  
1  aapl_2021-09-25  
2  aapl_2020-09-26  
3  aapl_2019-09-28  
4  aapl_2018-09-29  


In [5]:
print(pickle_dict['14.pkl'].iloc[0]['section14'])

 Item 14. Principal Accountant Fees and Services 

The information required by this Item will be included in the 2023 Proxy Statement, and is incorporated herein by reference. 

Apple Inc. | 2022 Form 10-K | 54 

PART IV 




In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 3000,
    chunk_overlap  = 300,
    length_function = len,
    add_start_index = True,
)



texts = []
for index, row in pickle_dict['1.pkl'].iterrows():
    texts.extend(text_splitter.create_documents([row['section1A']]))
    
# texts = text_splitter.create_documents([pickle_dict['1.pkl'].iloc[0]['section1A']])
print(texts[0])
print(texts[1])


page_content='Item 1. Business \n\nCompany Background \n\nThe Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company&#8217;s fiscal year is the 52- or 53-week period that ends on the last Saturday of September. \n\nProducts \n\niPhone \n\niPhone &#174; is the Company&#8217;s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 14 Pro, iPhone 14, iPhone 13, iPhone SE &#174; , iPhone 12 and iPhone 11. \n\nMac \n\nMac &#174; is the Company&#8217;s line of personal computers based on its macOS &#174; operating system. The Mac line includes laptops MacBook Air &#174; and MacBook Pro &#174; , as well as desktops iMac &#174; , Mac mini &#174; , Mac Studio&#8482; and Mac Pro &#174; . \n\niPad \n\niPad &#174; is the Company&#8217;s line of multipurpose tablets based on its iPadOS &#174; operating system. The iPad line includes iPad Pro &#174; , iPad Air 

In [7]:
len(texts)

6

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import LLMChain

# p = """
# As a capable financial analyst, your task is to review a section from Apple Inc.'s latest SEC filing.
# This section contains information about the company's financial position, its recent performance, and future plans.
# Please create a list of questions that could be answered by the content of the filing, along with their respective answers.
# These questions should focus on key points such as financial health, risk factors, executive compensation, business operations, and forward-looking statements.
# Provide concise, clear answers for each question based on the information presented in the filing.

# {filing_content}
# """

p_v2 = """
As a capable financial analyst, your task is to review a section from Apple Inc.'s latest SEC filing.
This section contains specific financial data, including key details about the company's revenue, sources of income, and profitability.
Generate a series of time-sensitive questions that specifically reference the concrete numbers and data points within the filing related to Apple's revenue streams.
For each question, provide a corresponding answer that is directly informed by the specific data in the document.
The question-answer pairs should cover areas such as revenue growth, breakdown of revenue by product or service, the impact of external factors on revenue, and how revenue performance might influence future strategy.

Use the following format for output:
```json
[
  {{"question": "XXX", "answer": "YYY"}}
  ...
]
```

{filing_content}
"""


import json

import openai


def run_anthropic(system_msg, user_content):
    return False


def run_gpt(system_msg,user_content,model="gpt-4",temperature=0.5):
    openai.api_key = os.environ['OPENAI_API_KEY']
    try:
        chat_completion = openai.ChatCompletion.create(
            # more info at https://platform.openai.com/docs/api-reference/completions/create
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_msg},
                {
                    "role": "user",
                    "content": user_content,
                },
            ],
        )
        gpt_responce = chat_completion.choices[0].message.content
        return gpt_responce
    except Exception as e:
        print(e)
        return False

def test_json(text):
    try:
        text = json.loads(text)
        return text
    except Exception as e:
        print(e,e.args)
        return False

def is_valid_json(json_string):
    try:
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False



def llm_extract_json(system_msg=None, user_content=None, model_seq=["GPT3", "GPT4"], attempts_each_model=3):
    model_select = {
        'Anthropic': {'settings': {}, 'func': run_anthropic},
        'GPT3': {'settings': {'model': "gpt-3.5-turbo", 'temperature': 0}, 'func': run_gpt},
        'GPT4': {'settings': {'model': "gpt-4", 'temperature': 0}, 'func': run_gpt}
    }
    if user_content:
        if system_msg is None:
            system_msg = """
            Output should be only a valid JSON object. Don't include any other text or explanation.
            Json:
            """
        for model in model_seq:
            if model in ['GPT4', 'GPT3']:
                for i in range(attempts_each_model):
                    model_name= model_select[model]['settings']['model']
                    temperature = model_select[model]['settings']['temperature'] + float((i/attempts_each_model))
                    print('RUNNING',i,model,temperature)
                    text = model_select[model]['func'](system_msg, user_content, model_name, round(temperature, 2))
                    if text:
                        text = test_json(text)
                        if text:
                            return text
                continue
            else:
                for i in range(attempts_each_model):
                    # print('RUNNING',model,i)
                    text = model_select[model]['func'](system_msg, user_content)
                    text = text.replace("```json", "")
                    text = text.replace("```", "")
                    if text:
                        print(text)
                        text = test_json(text)
                        if text:
                            return text
                continue
    return False

                
def generate_qa_pairs_from_filing(filing_content):
    llm = ChatOpenAI(temperature=0.1, model_name='gpt-4')
    # llm = OpenAI(temperature=0.1)
    prompt = PromptTemplate(
        input_variables=["filing_content"],
        template = p_v2
      )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(filing_content)
    print(result)
    if not is_valid_json(result):
        print("working")
        result = llm_extract_json(system_msg=None,user_content=result,model_seq=["GPT3","GPT4"],attempts_each_model=3)
    print(result)
    return result

In [12]:
result = generate_qa_pairs_from_filing(texts[0].page_content)

```json
[
  {"question": "What are the main product lines of Apple Inc. as mentioned in the 2022 Form 10-K?", "answer": "The main product lines of Apple Inc. include iPhone, Mac, iPad, and Wearables, Home and Accessories. The iPhone line includes models like iPhone 14 Pro, iPhone 14, iPhone 13, iPhone SE, iPhone 12 and iPhone 11. The Mac line includes laptops like MacBook Air and MacBook Pro, as well as desktops like iMac, Mac mini, Mac Studio and Mac Pro. The iPad line includes iPad Pro, iPad Air, iPad and iPad mini. The Wearables, Home and Accessories line includes products like AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories."},
  {"question": "What services does Apple Inc. offer as per the 2022 Form 10-K?", "answer": "Apple Inc. offers services like Advertising, AppleCare, Cloud Services, and Digital Content. The Advertising services include various third-party licensing arrangements and the Company's own advertising platforms. AppleCare is a portfolio 

In [10]:
json.dumps(result)

'{"error": "No financial data found"}'

In [22]:
result_lst = []
for i, t in enumerate(texts):
    result_lst.extend(generate_qa_pairs_from_filing(t.page_content))
    
    
with open("result.json", "w") as f:
    json.dump(result_lst, f)

Apologies for the confusion, but the provided text does not contain specific financial data or concrete numbers related to Apple's revenue streams. To generate the requested question-answer pairs, please provide a section from Apple Inc.'s latest SEC filing that includes specific financial figures such as revenue, net income, operating expenses, etc.
working
RUNNING 0 GPT3 0.0
{'question': "What was Apple's revenue in the latest SEC filing?", 'answer': '$100 billion'}
```json
[
  {"question": "What platforms does Apple operate for customers to discover and download applications and digital content?", "answer": "Apple operates various platforms including the App Store, which allows customers to discover and download applications and digital content such as books, music, video, games and podcasts."},
  {"question": "What are some of the subscription-based services offered by Apple?", "answer": "Apple offers subscription-based services including Apple Arcade, a game subscription service; 

{'data': [{'question': 'What new iPhone models were released by Apple in 2020 and 2021?', 'answer': 'In 2020, Apple released iPhone 12, iPhone 12 mini, iPhone 12 Pro and iPhone 12 Pro Max. In 2021, the company released iPhone 13, iPhone 13 mini, iPhone 13 Pro and iPhone 13 Pro Max.'}, {'question': 'What new Mac models were released by Apple in 2020 and 2021?', 'answer': 'In November 2020, Apple released new versions of MacBook Air, 13-inch MacBook Pro and Mac mini. In May 2021, the company released a redesigned iMac. In October 2021, Apple released a redesigned MacBook Pro, available in 14- and 16-inch models.'}, {'question': 'What new iPad models were released by Apple in 2020 and 2021?', 'answer': 'In October 2020, Apple released a new iPad Air. In April 2021, the company released a new iPad Pro powered by the Apple M1 chip. In September 2021, Apple released an updated iPad and a new iPad mini.'}, {'question': "What products are included in Apple's Wearables, Home and Accessories lin

```json
[
  {"question": "What are the primary markets for Apple's products and services?", "answer": "Apple's primary markets are the consumer, small and mid-sized business, education, enterprise and government markets."},
  {"question": "How does Apple distribute its products?", "answer": "Apple distributes its products directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. It also uses indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers."},
  {"question": "What percentage of Apple's total net sales in 2020 came from direct and indirect distribution channels?", "answer": "In 2020, Apple's net sales through its direct and indirect distribution channels accounted for 34% and 66%, respectively, of total net sales."},
  {"question": "Did any single customer account for more than 10% of net sales in 2020, 2019 a

[{'question': "What was the increase in Apple's net sales in 2019 compared to 2018?", 'answer': "Apple's net sales increased by $5.4 billion or 2% during 2019 compared to 2018."}, {'question': 'What were the main contributors to the increase in net sales during 2019?', 'answer': 'The increase in net sales during 2019 was primarily due to higher sales of Services and Wearables, Home and Accessories.'}, {'question': 'What was the net sales from iPhone in 2019?', 'answer': 'The net sales from iPhone in 2019 was $142,381 million.'}, {'question': 'What was the net sales from Services in 2019?', 'answer': 'The net sales from Services in 2019 was $46,291 million.'}, {'question': 'What was the total net sales in 2019?', 'answer': 'The total net sales in 2019 was $259,764 million.'}, {'question': "Which region contributed the most to Apple's net sales in 2019?", 'answer': "The Americas region contributed the most to Apple's net sales in 2019 with $116,896 million."}, {'question': "How did the s

The provided text does not contain specific financial data or concrete numbers related to Apple's revenue streams. Please provide the relevant financial data or SEC filing section to generate the required question-answer pairs.
working
RUNNING 0 GPT3 0.0
{'error': 'No relevant financial data or SEC filing section provided.'}
Sorry, but I can't assist with that.
working
RUNNING 0 GPT3 0.0
{'message': "Sorry, but I can't assist with that."}
watchOS is the Company&#8217;s operating system for Apple Watch. watchOS 5, released in September 2018, includes new features such as Walkie-Talkie, Podcasts and enhanced fitness and health capabilities. 

Services 

Services include the Company&#8217;s digital content stores and streaming services, AppleCare, Apple Pay, licensing and other services. The Company&#8217;s digital content stores and streaming services include the iTunes Store, App Store, Mac App Store, TV App, Book Store and Apple Music. AppleCare offers a range of support options for th

[{'question': 'What are the fee-based services that Apple offers to extend phone support eligibility and hardware repairs?', 'answer': 'Apple offers AppleCare + (AC+) and the AppleCare Protection Plan (APP) as fee-based services to extend phone support eligibility and hardware repairs.'}, {'question': 'What additional coverage does AppleCare + (AC+) offer?', 'answer': 'AppleCare + (AC+) offers additional coverage for instances of accidental damage and is available in certain countries for certain products. Additionally, AC+ with theft and loss protection is available for iPhone in the U.S.'}, {'question': 'What is Apple Pay and what does it support?', 'answer': "Apple Pay is Apple's cashless payment service available in certain countries. It allows users to pay for purchases in participating stores accepting contactless payments and to pay for purchases within participating apps on qualifying devices. It accepts credit and debit cards across major card networks and also supports reward

TypeError: 'bool' object is not iterable

In [ ]:
result = llm_extract_json(system_msg=None,user_content=result,model_seq=["GPT3","GPT4"],attempts_each_model=3)